In [1]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [2]:
import ee
#ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
import geemap
# geemap.update_package()
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [10]:
# get list of cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/geo_ref.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM2,COD,ADM2
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Extract OSM by city

In [9]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name'] + '-' + boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    #query OSM
    ## create bounding box
    bb = boundary_geo_ee.union(1).geometry().buffer(100).bounds(1000)
    print(bb.getInfo())
    ## return the list of coordinates
    listCoords = ee.Array.cat(bb.coordinates(), 1); 
    ##Casting it to an array makes it possible to slice out the x and y coordinates:
    ##get the X-coordinates
    xCoords = listCoords.slice(1, 0, 1); #print('xCoords', xCoords.getInfo());
    yCoords = listCoords.slice(1, 1, 2); #print('yCoords', yCoords.getInfo());
    ## Reducing each array reveals then the min and max values:
    ## reduce the arrays to find the max (or min) value
    West = ee.Number(xCoords.reduce('min', [0]).get([0,0])).getInfo(); 
    East = ee.Number(xCoords.reduce('max', [0]).get([0,0])).getInfo(); 
    North = ee.Number(yCoords.reduce('min', [0]).get([0,0])).getInfo(); 
    South = ee.Number(yCoords.reduce('max', [0]).get([0,0])).getInfo(); 
    
    ## set OSM tags relevant to open space category
    tags = {'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} #{'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} #}

    # use bounding box to get geodataframe of all OSM data on recreation sites/parks. Use .drop to remove properties that have incompatible names with FeatureCollections
    RecSites = geemap.osm_gdf_from_bbox(North, South, East, West, tags)#.drop(columns=['contact:p.o.box']) # use to remove properties that have incompatible names with FeatureCollections
    RecSites = RecSites.reset_index()
    #RecSites.sort_index()#.sample(3)
    #RecSites.sort_values(by=['osmid'],axis='index')
    
    ## keep only columns desired to reduce file size 
    RecSites = RecSites[RecSites['element_type']!= 'node']
    RecSites = RecSites[RecSites.geom_type != 'LineString']
    RecSites = RecSites.loc[:, ['osmid','geometry']] #['element_type','osmid','leisure','boundary','name','geometry']
    #RecSites#.sample(3)
    
    # convert Geodataframe to GeoJson
    RecSitesGJ = geemap.gdf_to_geojson(RecSites)
    
    # convert GeoJson to ee.FeatureCollection
    RecSitesFC = geemap.geojson_to_ee(RecSitesGJ)#.filter(ee.Filter.eq('element_type','way')).select(['osmid','element_type','geometry','leisure','boundary'])
    
    # Download FeatureCollection of OSM data (ways/polygons only) to Google Drive as geojson
    # Set configuration parameters for output vector
    task_config = {
        'folder': 'data',  # output Google Drive folder
        'fileFormat': 'Geojson',
        'description': boundary_id + '-OSM-OpenSpace',
        #'selectors': col_names,  # a list of properties/attributes to be exported
    }

    print('Exporting ' +boundary_id+ '-OSM-OpenSpace')
    task = ee.batch.Export.table.toDrive(RecSitesFC, **task_config)
    task.start()

0
BRA-Salvador-ADM4union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-38.6479090706965, -13.018044596390993], [-38.303629621586396, -13.018044596390993], [-38.303629621586396, -12.755388555364869], [-38.6479090706965, -12.755388555364869], [-38.6479090706965, -13.018044596390993]]]}


/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


ExportingBRA-Salvador-ADM4union-OSM-OpenSpace
1
COD-Bukavu-ADM3union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[28.792793548602692, -2.5620678433716595], [28.893998020672637, -2.5620678433716595], [28.893998020672637, -2.4592572502108943], [28.792793548602692, -2.4592572502108943], [28.792793548602692, -2.5620678433716595]]]}
ExportingCOD-Bukavu-ADM3union-OSM-OpenSpace
2
COD-Uvira-ADM3union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[28.82714294281798, -3.659985483396138], [29.235821625356785, -3.659985483396138], [29.235821625356785, -3.2694187180170435], [28.82714294281798, -3.2694187180170435], [28.82714294281798, -3.659985483396138]]]}
ExportingCOD-Uvira-ADM3union-OSM-OpenSpace
3
COG-Brazzaville-ADM4union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[15.144029707041506, -4.376178909213759], [15.321870372221882, -4.376178909213759], [15.321870372221882, -4.075183005612928], [15.144029707041506, -4.075183005612928], [15.144029707041506, -4.376178

/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


ExportingCOL-Barranquilla-ADM4union-OSM-OpenSpace
5
ETH-Addis_Ababa-ADM4union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[38.63788144391243, 8.832375833352112], [38.907203886081845, 8.832375833352112], [38.907203886081845, 9.100240868864804], [38.63788144391243, 9.100240868864804], [38.63788144391243, 8.832375833352112]]]}
ExportingETH-Addis_Ababa-ADM4union-OSM-OpenSpace
6
ETH-Dire_Dawa-ADM3union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[41.63224928538966, 9.444486174080263], [42.42288500867382, 9.444486174080263], [42.42288500867382, 9.842502079164673], [41.63224928538966, 9.842502079164673], [41.63224928538966, 9.444486174080263]]]}
ExportingETH-Dire_Dawa-ADM3union-OSM-OpenSpace
7
KEN-Nairobi-ADM3union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[36.66356432482584, -1.4465086196452817], [37.10587899739047, -1.4465086196452817], [37.10587899739047, -1.1596843456754946], [36.66356432482584, -1.1596843456754946], [36.66356432482584, -1.44650861964

/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


ExportingMEX-Mexico_City-ADM2union-OSM-OpenSpace
10
MEX-Monterrey-ADM2union
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-100.86290013026063, 25.222887827281006], [-99.68520109085905, 25.222887827281006], [-99.68520109085905, 26.396066640281617], [-100.86290013026063, 26.396066640281617], [-100.86290013026063, 25.222887827281006]]]}


/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/srv/conda/envs/notebook/lib/python3.8/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


ExportingMEX-Monterrey-ADM2union-OSM-OpenSpace


# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [11]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-cities4forests' 

In [12]:
out_dir = os.getcwd()

In [14]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-OSM-OpenSpace.geojson'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/open_space/openstreetmap/v_0/'+ boundary_id + '-OSM-open_space-2022.geojson',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Salvador-ADM4union

 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: MEX-Mexico_City-ADM2union

 boundary_id: MEX-Monterrey-ADM2union


# Visualize

In [5]:
## create map
Map = geemap.Map(height="350px")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [17]:
i = 10
boundary_id = boundary_georef.loc[i, 'geo_name'] + '-' + boundary_georef.loc[i, 'aoi_boundary_name']
print(boundary_id)

MEX-Monterrey-ADM2union


In [18]:
# read boundaries
boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
boundary_geo = requests.get(boundary_path).json()
boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)

In [19]:
#read open space
openspace_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/open_space/openstreetmap/v_0/'+boundary_id+'-OSM-open_space-2022.geojson'
openspace_geo = requests.get(openspace_path).json()
openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)

In [20]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=12)

In [21]:
Map.addLayer(boundary_geo_ee, {'color': 'FF0000'}, 'City boundary')

In [22]:
Map.addLayer(openspace_geo_ee,  {'color': 'FF00FF'}, 'Open space (OSM)')